In [1]:
from pyspark.sql import SparkSession,functions as F
from pyspark import StorageLevel
from pyspark import SparkConf
from pyspark import SparkConf
import sys 
sys.path.append("/home/john_user/pyspark_learning/code/")
from utility.utils import return_files_list as rfl

conf = SparkConf()

conf.set("spark.app.name", "ComprehensiveSparkJob") 
conf.set("spark.master", "local[*]")                
conf.set("spark.driver.memory", "4g")      
conf.set("spark.driver.cores", "1")                 
conf.set("spark.ui.port", "4040")                

# Executor Settings
conf.set("spark.executor.memory", "2g")      
conf.set("spark.executor.cores", "2")               
conf.set("spark.executor.instances", "3")         


conf.set("spark.default.parallelism", "6")         
conf.set("spark.sql.shuffle.partitions", "6")       
conf.set("spark.task.cpus", "1")  

# Data Handling Settings
conf.set("spark.memory.fraction", "0.8")            
conf.set("spark.memory.storageFraction", "0.5")     

# Serialization Settings
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")  
conf.set("spark.kryo.registrationRequired", "true")  
conf.set("spark.kryo.classesToRegister", "org.apache.spark.sql.Row")  

# Debugging and Logging
conf.set("spark.eventLog.enabled", "true")           
conf.set("spark.eventLog.dir", "/root/spark_log/spark-events/")  
conf.set("spark.history.fs.logDirectory", "/root/spark_log/spark-history/")  


In [2]:
spark = SparkSession.builder.config(conf=conf).getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/01 05:32:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
input_json_file_path = "/root/docker_dataset/h1b_data/"  
file_pattern = r'.*\.(csv|xlsx)$'
input_file_list  = rfl(input_json_file_path,file_pattern)
# parquet_file_path = "/root/docker_dataset/json_splits/table_example_1.parquet"



In [4]:

def func_cache_dft_data(dataframe) :
    dataframe.persist(StorageLevel.MEMORY_AND_DISK)

def func_json_dataframe (input_file:str) :
    return  spark.read.option("multiline", "true").json(input_file)

def func_repartion_dataframe(dataframe, repartion_count) :
    return dataframe.repartition(repartion_count)
 

def func_explode_based_on_one_key (input_dataframe,explode_key:str , alias_of_column :str)  :
    return  input_dataframe.select(F.explode(F.col(explode_key)).alias(alias_of_column))

def func_explode_based_on_depper_root_key (input_dataframe,explode_key:list  , drop_column :str)  :
    """ Always try to assemeble  he json key  based on the root level  in heirarchial order like root -->values-->payload-->commits then the explode_key   value should be [root,values,payload,commits]"""
    column_level_analysis = (".").join(explode_key)
    return  input_dataframe.select(f"{explode_key[0]}.*",F.explode_outer(F.col(column_level_analysis)).alias(column_level_analysis.replace(".","_"))).drop(drop_column)


def func_get_rdd_num_partitions( dataframe) :
    return dataframe.rdd.getNumPartitions()

def func_write_data_to_parquet ( file_path, dataframe,write_mode) :
    dataframe.write.parquet(file_path,mode=write_mode)


def func_read_parquet_file(file_path) :
    return spark.read.parquet(file_path)

def func_stop_spark () :
      spark.stop()


def func_remane_column_names (dataframe) :
    column_list = [F.col(c).alias(c.replace('.', '_')) for c in dataframe.columns]
    for   values in column_list:
        print(values)
    return  dataframe.select([F.col(c).alias(c.replace('.', '_')) for c in dataframe.columns])


def  func_limit_rows_dataframe (dataframe, limit_row_count) :
    return dataframe. limit(limit_row_count)

def  finc_count_dft_rows(dataframe):
     return dataframe.count()



In [5]:

def func_main () :
     processed_dft_row_count= 0 
     for file_list in input_file_list :
         big_dpart_dft = func_json_dataframe (input_json_file_path+file_list)
         big_dpart_dft = func_repartion_dataframe(big_dpart_dft,100)
         big_dpart_dft_json_normalised = func_explode_based_on_one_key (big_dpart_dft,"values","table_example")
         big_dpart_dft_json_normalised = func_explode_based_on_depper_root_key(big_dpart_dft_json_normalised,["table_example","entities"],"entities")
         func_cache_dft_data(big_dpart_dft_json_normalised)
         print(func_get_rdd_num_partitions(big_dpart_dft_json_normalised))
         big_dpart_dft_json_norm_renamed  = func_remane_column_names(big_dpart_dft_json_normalised)
         processed_dft_row_count += finc_count_dft_rows(big_dpart_dft_json_norm_renamed)
         func_write_data_to_parquet(parquet_file_path,big_dpart_dft_json_norm_renamed,"append")
     return processed_dft_row_count

In [ ]:
try :
    # json_input_row_count  = func_main () 
    big_dpart_parquet = func_read_parquet_file(parquet_file_path)
    # parquet_row_count  = finc_count_dft_rows(big_dpart_parquet)
    # big_dpart_parquet = func_limit_rows_dataframe(big_dpart_parquet,100)
    big_dpart_parquet.show()
except  Exception as error :
    spark.stop()
    raise error 

In [ ]:
db_url = "jdbc:postgresql://host.docker.internal:5432/crime_data_la"
db_properties = {
    "user": "john_user",
    "password": "abc@12345"
}

table_name = "crime_data.transactions_data"
df_table = spark.read.jdbc(url=db_url, table=table_name, properties=db_properties)
df_table.show()


In [18]:
data_type  = """
cast(CASE_NUMBER as varchar(450) ) as CASE_NUMBER, cast(CASE_STATUS as varchar(100)) as CASE_STATUS, cast(RECEIVED_DATE as date ) as RECEIVED_DATE, cast(DECISION_DATE as date ) as DECISION_DATE, cast(ORIGINAL_CERT_DATE as date  ) as ORIGINAL_CERT_DATE, cast(VISA_CLASS as  varchar(50) ) as VISA_CLASS, cast(JOB_TITLE as  varchar(850)) as JOB_TITLE, cast(SOC_CODE as varchar(100) ) as SOC_CODE, cast(SOC_TITLE as varchar(850) ) as SOC_TITLE, cast(FULL_TIME_POSITION as varchar(1) ) as FULL_TIME_POSITION, cast(BEGIN_DATE as  date ) as BEGIN_DATE, cast(END_DATE as date  ) as END_DATE, cast(TOTAL_WORKER_POSITIONS as integer   ) as TOTAL_WORKER_POSITIONS, cast(NEW_EMPLOYMENT as  integer) as NEW_EMPLOYMENT, cast(CONTINUED_EMPLOYMENT as integer ) as CONTINUED_EMPLOYMENT, cast(CHANGE_PREVIOUS_EMPLOYMENT as integer ) as CHANGE_PREVIOUS_EMPLOYMENT, cast(NEW_CONCURRENT_EMPLOYMENT as integer ) as NEW_CONCURRENT_EMPLOYMENT, cast(CHANGE_EMPLOYER as  integer) as CHANGE_EMPLOYER, cast(AMENDED_PETITION as  integer) as AMENDED_PETITION, cast(EMPLOYER_NAME as  varchar(850) ) as EMPLOYER_NAME, cast(TRADE_NAME_DBA as string ) as TRADE_NAME_DBA, cast(EMPLOYER_ADDRESS1 as string ) as EMPLOYER_ADDRESS1, cast(EMPLOYER_ADDRESS2 as string ) as EMPLOYER_ADDRESS2, cast(EMPLOYER_CITY as  string) as EMPLOYER_CITY, cast(EMPLOYER_STATE as string ) as EMPLOYER_STATE, cast(EMPLOYER_POSTAL_CODE as  string) as EMPLOYER_POSTAL_CODE, cast(EMPLOYER_COUNTRY as string ) as EMPLOYER_COUNTRY, cast(EMPLOYER_PROVINCE as string ) as EMPLOYER_PROVINCE, cast(EMPLOYER_PHONE as string ) as EMPLOYER_PHONE, cast(EMPLOYER_PHONE_EXT as  int) as EMPLOYER_PHONE_EXT, cast(NAICS_CODE as int  ) as NAICS_CODE, cast(EMPLOYER_POC_LAST_NAME as  string) as EMPLOYER_POC_LAST_NAME, cast(EMPLOYER_POC_FIRST_NAME as  string) as EMPLOYER_POC_FIRST_NAME, cast(EMPLOYER_POC_MIDDLE_NAME as string ) as EMPLOYER_POC_MIDDLE_NAME, cast(EMPLOYER_POC_JOB_TITLE as string ) as EMPLOYER_POC_JOB_TITLE, cast(EMPLOYER_POC_ADDRESS1 as string ) as EMPLOYER_POC_ADDRESS1, cast(EMPLOYER_POC_ADDRESS2 as string) as EMPLOYER_POC_ADDRESS2, cast(EMPLOYER_POC_CITY as string ) as EMPLOYER_POC_CITY, cast(EMPLOYER_POC_STATE as  string) as EMPLOYER_POC_STATE, cast(EMPLOYER_POC_POSTAL_CODE as string ) as EMPLOYER_POC_POSTAL_CODE, cast(EMPLOYER_POC_COUNTRY as  string) as EMPLOYER_POC_COUNTRY, cast(EMPLOYER_POC_PROVINCE as string ) as EMPLOYER_POC_PROVINCE, cast(EMPLOYER_POC_PHONE as  string) as EMPLOYER_POC_PHONE, cast(EMPLOYER_POC_PHONE_EXT as  integer) as EMPLOYER_POC_PHONE_EXT, cast(EMPLOYER_POC_EMAIL as string ) as EMPLOYER_POC_EMAIL, cast(AGENT_REPRESENTING_EMPLOYER as string ) as AGENT_REPRESENTING_EMPLOYER, cast(AGENT_ATTORNEY_LAST_NAME as string ) as AGENT_ATTORNEY_LAST_NAME, cast(AGENT_ATTORNEY_FIRST_NAME as string ) as AGENT_ATTORNEY_FIRST_NAME, cast(AGENT_ATTORNEY_MIDDLE_NAME as string ) as AGENT_ATTORNEY_MIDDLE_NAME, cast(AGENT_ATTORNEY_ADDRESS1 as  string) as AGENT_ATTORNEY_ADDRESS1, cast(AGENT_ATTORNEY_ADDRESS2 as string ) as AGENT_ATTORNEY_ADDRESS2, cast(AGENT_ATTORNEY_CITY as string ) as AGENT_ATTORNEY_CITY, cast(AGENT_ATTORNEY_STATE as string ) as AGENT_ATTORNEY_STATE, cast(AGENT_ATTORNEY_POSTAL_CODE as  string) as AGENT_ATTORNEY_POSTAL_CODE, cast(AGENT_ATTORNEY_COUNTRY as string ) as AGENT_ATTORNEY_COUNTRY, cast(AGENT_ATTORNEY_PROVINCE as  string) as AGENT_ATTORNEY_PROVINCE, cast(AGENT_ATTORNEY_PHONE as  string) as AGENT_ATTORNEY_PHONE, cast(AGENT_ATTORNEY_PHONE_EXT as integer ) as AGENT_ATTORNEY_PHONE_EXT, cast(AGENT_ATTORNEY_EMAIL_ADDRESS as string ) as AGENT_ATTORNEY_EMAIL_ADDRESS, cast(LAWFIRM_NAME_BUSINESS_NAME as string ) as LAWFIRM_NAME_BUSINESS_NAME, cast(STATE_OF_HIGHEST_COURT as string ) as STATE_OF_HIGHEST_COURT, cast(NAME_OF_HIGHEST_STATE_COURT as  string) as NAME_OF_HIGHEST_STATE_COURT, cast(WORKSITE_WORKERS as string ) as WORKSITE_WORKERS, cast(SECONDARY_ENTITY as string ) as SECONDARY_ENTITY, cast(SECONDARY_ENTITY_BUSINESS_NAME as string  ) as SECONDARY_ENTITY_BUSINESS_NAME, cast(WORKSITE_ADDRESS1 as string ) as WORKSITE_ADDRESS1, cast(WORKSITE_ADDRESS2 as string ) as WORKSITE_ADDRESS2, cast(WORKSITE_CITY as  string) as WORKSITE_CITY, cast(WORKSITE_COUNTY as  string) as WORKSITE_COUNTY, cast(WORKSITE_STATE as  string) as WORKSITE_STATE, cast(WORKSITE_POSTAL_CODE as string ) as WORKSITE_POSTAL_CODE, cast(WAGE_RATE_OF_PAY_FROM as numeric  ) as WAGE_RATE_OF_PAY_FROM, cast(WAGE_RATE_OF_PAY_TO as numeric ) as WAGE_RATE_OF_PAY_TO, cast(WAGE_UNIT_OF_PAY as string ) as WAGE_UNIT_OF_PAY, cast(PREVAILING_WAGE as numeric ) as PREVAILING_WAGE, cast(PW_UNIT_OF_PAY as string ) as PW_UNIT_OF_PAY, cast(PW_TRACKING_NUMBER as string ) as PW_TRACKING_NUMBER, cast(PW_WAGE_LEVEL as  string) as PW_WAGE_LEVEL, cast(PW_OES_YEAR as int ) as PW_OES_YEAR, cast(PW_OTHER_SOURCE as  string) as PW_OTHER_SOURCE, cast(PW_OTHER_YEAR as int  ) as PW_OTHER_YEAR, cast(PW_SURVEY_PUBLISHER as string ) as PW_SURVEY_PUBLISHER, cast(PW_SURVEY_NAME as string ) as PW_SURVEY_NAME, cast(TOTAL_WORKSITE_LOCATIONS as string ) as TOTAL_WORKSITE_LOCATIONS, cast(AGREE_TO_LC_STATEMENT as string ) as AGREE_TO_LC_STATEMENT, cast(H_1B_DEPENDENT as string ) as H_1B_DEPENDENT, cast(WILLFUL_VIOLATOR as  string) as WILLFUL_VIOLATOR, cast(SUPPORT_H1B as string ) as SUPPORT_H1B, cast(STATUTORY_BASIS as string ) as STATUTORY_BASIS, cast(APPENDIX_A_ATTACHED as  string) as APPENDIX_A_ATTACHED, cast(PUBLIC_DISCLOSURE as string ) as PUBLIC_DISCLOSURE, cast(PREPARER_LAST_NAME as string ) as PREPARER_LAST_NAME, cast(PREPARER_FIRST_NAME as string ) as PREPARER_FIRST_NAME, cast(PREPARER_MIDDLE_INITIAL as string ) as PREPARER_MIDDLE_INITIAL, cast(PREPARER_BUSINESS_NAME as string ) as PREPARER_BUSINESS_NAME, cast(PREPARER_EMAIL as string ) as  PREPARER_EMAIL """

In [19]:
csv_path  = '/root/docker_dataset/h1b_data/Combined_LCA_Disclosure_Data_FY2020.csv'
formatted_data_type = [f'{item.strip()}' for item in data_type.split(',')]
print(formatted_data_type)
df_csv = spark.read.option("delimiter", ",").option("header", True).option("multiline", "true").csv(csv_path)
df_csv_updated = df_csv.selectExpr(
    formatted_data_type
)
df_csv_updated.show()

['cast(CASE_NUMBER as varchar(450) ) as CASE_NUMBER', 'cast(CASE_STATUS as varchar(100)) as CASE_STATUS', 'cast(RECEIVED_DATE as date ) as RECEIVED_DATE', 'cast(DECISION_DATE as date ) as DECISION_DATE', 'cast(ORIGINAL_CERT_DATE as date  ) as ORIGINAL_CERT_DATE', 'cast(VISA_CLASS as  varchar(50) ) as VISA_CLASS', 'cast(JOB_TITLE as  varchar(850)) as JOB_TITLE', 'cast(SOC_CODE as varchar(100) ) as SOC_CODE', 'cast(SOC_TITLE as varchar(850) ) as SOC_TITLE', 'cast(FULL_TIME_POSITION as varchar(1) ) as FULL_TIME_POSITION', 'cast(BEGIN_DATE as  date ) as BEGIN_DATE', 'cast(END_DATE as date  ) as END_DATE', 'cast(TOTAL_WORKER_POSITIONS as integer   ) as TOTAL_WORKER_POSITIONS', 'cast(NEW_EMPLOYMENT as  integer) as NEW_EMPLOYMENT', 'cast(CONTINUED_EMPLOYMENT as integer ) as CONTINUED_EMPLOYMENT', 'cast(CHANGE_PREVIOUS_EMPLOYMENT as integer ) as CHANGE_PREVIOUS_EMPLOYMENT', 'cast(NEW_CONCURRENT_EMPLOYMENT as integer ) as NEW_CONCURRENT_EMPLOYMENT', 'cast(CHANGE_EMPLOYER as  integer) as CHANGE_

25/02/01 05:42:25 WARN CharVarcharUtils: The Spark cast operator does not support char/varchar type and simply treats them as string type. Please use string type directly to avoid confusion. Otherwise, you can set spark.sql.legacy.charVarcharAsString to true, so that Spark treat them as string type as same as Spark 3.0 and earlier
25/02/01 05:42:25 WARN CharVarcharUtils: The Spark cast operator does not support char/varchar type and simply treats them as string type. Please use string type directly to avoid confusion. Otherwise, you can set spark.sql.legacy.charVarcharAsString to true, so that Spark treat them as string type as same as Spark 3.0 and earlier
25/02/01 05:42:25 WARN CharVarcharUtils: The Spark cast operator does not support char/varchar type and simply treats them as string type. Please use string type directly to avoid confusion. Otherwise, you can set spark.sql.legacy.charVarcharAsString to true, so that Spark treat them as string type as same as Spark 3.0 and earlier
2

+------------------+-----------+-------------+-------------+------------------+----------+--------------------+--------+--------------------+------------------+----------+----------+----------------------+--------------+--------------------+--------------------------+-------------------------+---------------+----------------+--------------------+--------------------+--------------------+-----------------+---------------+--------------+--------------------+--------------------+-----------------+--------------+------------------+----------+----------------------+-----------------------+------------------------+----------------------+---------------------+---------------------+-----------------+------------------+------------------------+--------------------+---------------------+------------------+----------------------+--------------------+---------------------------+------------------------+-------------------------+--------------------------+-----------------------+-------------------

In [22]:
df_csv_updated.select(df_csv_updated["CASE_STATUS"]).show()
df_csv_updated.count()

+-----------+
|CASE_STATUS|
+-----------+
|  Certified|
|  Certified|
|  Certified|
|  Certified|
|  Certified|
|  Certified|
|  Certified|
|  Certified|
|  Certified|
|  Certified|
|  Certified|
|  Certified|
|  Certified|
|  Certified|
|  Certified|
|  Certified|
|  Certified|
|  Certified|
|  Certified|
|  Certified|
+-----------+
only showing top 20 rows



577328

In [26]:
db_url = "jdbc:postgresql://host.docker.internal:5432/crime_data_la"
db_properties = {
    "user": "john_user",
    "password": "abc@12345"
}
mode = "overwrite"
table_name = "spark_project.h1b_data"
df_csv_updated.write.option('driver', 'org.postgresql.Driver').jdbc(url=db_url, table=table_name, properties=db_properties)
